<a href="https://colab.research.google.com/github/21jzele0449/CLI-Number-guessing-game/blob/main/Rag_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.9 MB/s eta 0:00:00


In [ ]:
%pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
from google.colab import files
from pinecone import Pinecone, ServerlessSpec
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from IPython.display import Markdown


In [ ]:
uploaded = files.upload()
pdf_file_name = list(uploaded.keys())[0]

Saving Usama Cv .pdf to Usama Cv .pdf


In [30]:
loader = PyPDFLoader(pdf_file_name)
documents = loader.load()

In [ ]:
from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)


In [ ]:
index_name = "cv-rag-project-01"
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
index = pc.Index(index_name)


In [ ]:
google_api_key = userdata.get('GOOGLE_API_KEY')

os.environ['GOOGLE_API_KEY'] = google_api_key


In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['3fd3f53a-5d00-4ead-a0d0-29b74ff2ac95']

In [ ]:
retriever = vector_store.as_retriever()
template = """Answer the question based only on the following context:
{context}.

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
api_key=userdata.get('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    # model="gemini-1.5-flash-8b",
    api_key=api_key,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
while True:
    query = input("Enter your question about my cv (or type 'exit' to quit): ")
    if query.lower() == 'exit':
        break
    try:
        result = rag_chain.invoke(query)
        display(Markdown(result))
    except Exception as e:
        print(f"An error occurred: {e}")

Enter your question about my cv (or type 'exit' to quit): what's my name


Usama Ahmad

Enter your question about my cv (or type 'exit' to quit): phone number


+923115421583

Enter your question about my cv (or type 'exit' to quit): qualification


BSc (Electrical Engineering) and Fsc (pre-Engineering).

Enter your question about my cv (or type 'exit' to quit): exit
